## Reference
- https://learn.deeplearning.ai/courses/building-code-agents-with-hugging-face-smolagents

## Overview: Secure Code Execution 
- This notebook shows how to perform secure code execution with `smolagents`, since coding agents carry more risk. It shows how to use:
    - Built in python interpreter. (still not 100% safe)
    - Secure/Remote Sandbox environment not in local env (cn be done with Docker / E2B sandbox)

## Show behavior of custom python interpreter

In [1]:
#!pip install git+https://github.com/huggingface/smolagents.git

In [2]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [3]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [4]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [5]:
# In our interpreter, it does not.
harmful_command="!echo Bad command"
run_capture_exception(harmful_command)

ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [6]:
# White list of imports
[
    're',
    'queue',
    'random',
    'statistics',
    'unicodedata',
    'itertools',
    'math',
    'stat',
    'time',
    'datetime',
    'collections',
    'numpy'
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy']

In [7]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['queue', 'statistics', 'itertools', 'math', 'stat', 'numpy', 'collections', 'time', 'random', 're', 'datetime', 'unicodedata']


In [8]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [9]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [10]:
# Can fill the drive with malicious code

custom_executor = LocalPythonExecutor(["PIL"])

harmful_command="""
from PIL import Image

img = Image.new('RGB', (100, 100), color='blue')

i=0
while i < 10000:
    img.save('simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)
# Let's not execute this but it would not error out, and it would bloat your system with images.

## Running in a sandbox

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [12]:
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5
)

Initializing executor, hold on...

Collecting smolagents

  Downloading smolagents-1.22.0-py3-none-any.whl.metadata (16 kB)

Collecting huggingface-hub>=0.31.2 (from smolagents)
  Downloading huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)

Requirement already satisfied: requests>=2.32.3 in /usr/local/lib/python3.12/site-packages (from smolagents) 
(2.32.4)
Requirement already satisfied: rich>=13.9.4 in /usr/local/lib/python3.12/site-packages (from smolagents) (14.1.0)
Requirement already satisfied: jinja2>=3.1.4 in /usr/local/lib/python3.12/site-packages (from smolagents) (3.1.6)
Requirement already satisfied: pillow>=10.0.1 in /usr/local/lib/python3.12/site-packages (from smolagents) (11.3.0)
Collecting python-dotenv (from smolagents)

  Downloading python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Collecting filelock (from huggingface-hub>=0.31.2->smolagents)

  Downloading filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)

Collecting fsspec>=2023.5.0 (from huggingface-hub>=0.31.2->smolagents)
  Downloading fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)

Requirement already satisfied: packaging>=20.9 in /usr/local/lib/python3.12/site-packages (from 
huggingface-hub>=0.31.2->smolagents) (25.0)
Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.12/site-packages (from 
huggingface-hub>=0.31.2->smolagents) (6.0.2)
Requirement already satisfied: tqdm>=4.42.1 in /usr/local/lib/python3.12/site-packages (from 
huggingface-hub>=0.31.2->smolagents) (4.67.1)
Requirement already satisfied: typing-extensions>=3.7.4.3 in /usr/local/lib/python3.12/site-packages (from 
huggingface-hub>=0.31.2->smolagents) (4.14.1)

Collecting hf-xet<2.0.0,>=1.1.3 (from huggingface-hub>=0.31.2->smolagents)
  Downloading hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.7 kB)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.12/site-packages (from 
jinja2>=3.1.4->smolagents) (3.0.2)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/site-packages (from 
requests>=2.32.3->smolagents) (3.4.3)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/site-packages (from 
requests>=2.32.3->smolagents) (3.10)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/site-packages (from 
requests>=2.32.3->smolagents) (2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/site-packages (from 
requests>=2.32.3->smolagents) (2025.8.3)
Requirement already satisfied: markdown-it-py>=2.2.0 in /usr/local/lib/python3.12/site-packages (from 
rich>=13.9.4->smolagents) (4.0.0)
Requirement already satisfied: pygments<3.0.0,>=2.13.0 in /usr/local/lib/python3.12/site-packages (from 
rich>=13.9.4->smolagents) (2.19.2)
Requirement already satisfied: mdurl~=0.1 in /usr/local/lib/python3.12/site-packages (from 
markdown-it-py>=2.2.0->rich>=13.9.4->smolagents) (0.1.2)

Downloading smolagents-1.22.0-py3-none-any.whl (149 kB)

Downloading huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
[?25l   [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/564.3 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m564.3/564.3 kB[0m [31m72.9 MB/s[0m eta [36m0:00:00[0m
[?25hDownloading python_dotenv-1.1.1-py3-none-any.whl (20 kB)
Downloading fsspec-2025.9.0-py3-none-any.whl (199 kB)

Downloading hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
[?25l   [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/3.2 MB[0m [31m?[0m eta [36m-:--:--[0m[2K 
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m3.2/3.2 MB[0m [31m147.4 MB/s[0m eta [36m0:00:00[0m
[?25hDownloading filelock-3.20.0-py3-none-any.whl (16 kB)

Installing collected packages: python-dotenv, hf-xet, fsspec, filelock, huggingface-hub, smolagents

Successfully installed filelock-3.20.0 fsspec-2025.9.0 hf-xet-1.1.10 huggingface-hub-0.35.3 python-dotenv-1.1.1 
smolagents-1.22.0


E2B is running

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> The agent is not always successful in this run.
</div>


In [13]:
output = agent.run(
    "Give me one of the top github repos from organization huggingface."
)
print("E2B executor result:", output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me one of the top github repos from organization huggingface.                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

Requirement already satisfied: requests in /usr/local/lib/python3.12/site-packages (2.32.4)

Collecting markdownify

  Downloading markdownify-1.2.0-py3-none-any.whl.metadata (9.9 kB)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/site-packages (from requests) 
(3.4.3)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/site-packages (from requests) (3.10)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/site-packages (from requests) 
(2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/site-packages (from requests) 
(2025.8.3)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.12/site-packages (from markdownify)
(4.13.4)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.12/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.7)
Requirement already satisfied: typing-extensions>=4.0.0 in /usr/local/lib/python3.12/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (4.14.1)
Downloading markdownify-1.2.0-py3-none-any.whl (15 kB)

Installing collected packages: markdownify
Successfully installed markdownify-1.2.0
[33mWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager, possibly rendering your system unusable. It is recommended to use a virtual environment 
instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and 
want to suppress this warning.[0m[33m
[0m

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
404 Client Error: Not Found for url: 
http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_inference_provider/hf-inference/models/Qwen/Qwen2.5-Co
der-32B-Instruct/v1/chat/completions (Request ID: 
Self=1-68f2d889-12acda7d5bf2552b2bd0de28;Root=1-68f2d889-6a19b8ba0067d4927bb4de85;385460d4-6101-45da-873a-e60086cdf
e9a)

[Step 1: Duration 0.30 seconds]

AgentGenerationError: Error in generating model output:
404 Client Error: Not Found for url: http://jupyter-api-proxy.internal.dlai/rev-proxy/huggingface_inference_provider/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Self=1-68f2d889-12acda7d5bf2552b2bd0de28;Root=1-68f2d889-6a19b8ba0067d4927bb4de85;385460d4-6101-45da-873a-e60086cdfe9a)